In [1]:
import os
import pandas as pd
from langchain.prompts import PromptTemplate
from huggingface_hub import InferenceClient

In [2]:
data_dir = "../202401_NFs"
dataframes = {}

def load_csvs():
    global dataframes
    dataframes = {}
    for filename in os.listdir(data_dir):
        if filename.endswith(".csv"):
            df_name = os.path.splitext(filename)[0]
            df_path = os.path.join(data_dir, filename)
            dataframes[df_name] = pd.read_csv(df_path)
    print(f"CSVs carregados: {list(dataframes.keys())}")

load_csvs()

CSVs carregados: ['202401_NFs_Cabecalho', '202401_NFs_Itens']


In [3]:
def generate_context():
    context = ""
    for name, df in dataframes.items():
        context += f"\nDataset: {name}\n"
        context += f"Colunas: {', '.join(df.columns)}\n"
        context += f"Total de linhas: {len(df)}\n"
    return context

In [11]:
template = """
Você é um assistente Python que ajuda a responder perguntas sobre múltiplos datasets carregados como DataFrames Pandas.

Aqui está um resumo dos datasets disponíveis:

{dataset_context}

Pergunta do usuário:
{user_question}
"""

prompt_template = PromptTemplate.from_template(template)

In [ ]:
client = InferenceClient(
    provider="nebius",
    api_key= os.environ["hf_api_key"],
)

In [13]:
def ask_question(user_question):
    context = generate_context()
    prompt = prompt_template.format(
        dataset_context=context,
        user_question=user_question
    )

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

    return completion.choices[0].message.content

In [15]:
answer = ask_question("me resuma o conteudo dos arquivos csv")

print(answer)

Certamente!

Aqui está um resumo dos dados nos arquivos CSV:

**Dataset: 202401_NFs_Cabecalho**

Este dataset contém informações sobre as notas fiscais, que são básicas, como:

* Chave de acesso
* Modelo
* Série
* Número
* Natureza da operação
* Data e emissão
* Evento mais recente
* Data/hora do evento mais recente
* Informações do emitente (CPF/CNPJ, Razão Social, Inscrição Estadual, UF, Município)
* Informações do destinatário (CNPJ, Nome, UF, Indicador IE)
* Outras informações (Destino da operação, Consumidor final, Presença do comprador)

Ele contém 100 linhas.

**Dataset: 202401_NFs_Itens**

Este dataset contém detalhes sobre os itens das notas fiscais, como:

* Chave de acesso
* Modelo
* Série
* Número
* Natureza da operação
* Data e emissão
* Informações do emitente (CPF/CNPJ, Razão Social, Inscrição Estadual, UF, Município)
* Informações do destinatário (CNPJ, Nome, UF, Indicador IE)
* Informações do item (Número do produto, Descrição do produto/serviço, Código NCM/SH, NCM/SH 